In [1]:
setwd(paste0(Sys.getenv('R_SOURCES'), '/house_prices'))
getwd()

[1] "/mnt/hdd/develop/kaggle/R_sources/house_prices"

In [13]:
source('main.R')

house_prices$helpers$import_libs()

combined_dataset <- house_prices$helpers$load_data()

numeric_data <- 
    combined_dataset %>% 
    select(house_prices$helpers$get_numeric_colnames(combined_dataset), -Id) %>%
    select(-one_of(house_prices$attributes_selection$discared_num_vars))

numeric_data %>% colnames %>% sort

[1] "BedroomAbvGr" "BsmtFinSF1"   "BsmtFullBath" "BsmtUnfSF"    "Fireplaces"  
 [6] "FullBath"     "GarageArea"   "GarageCars"   "GarageYrBlt"  "GrLivArea"   
[11] "HalfBath"     "LotArea"      "LotFrontage"  "MasVnrArea"   "MoSold"      
[16] "OpenPorchSF"  "OverallCond"  "OverallQual"  "SalePrice"    "TotalBsmtSF" 
[21] "TotRmsAbvGrd" "WoodDeckSF"   "X1stFlrSF"    "X2ndFlrSF"    "YearBuilt"   
[26] "YearRemodAdd" "YrSold"

In [ ]:
numeric_data %>%
gather(var, x) %>%
mutate(
    log = log(x + 1),
    sqrt = sqrt(x)
) %>%
gather(predictor, value, -var) %>%
filter(!is.na(value)) %>%
group_by(var, predictor) %>%
mutate(
    value_normed = (value - mean(value)) / sd(value)
) %>%
group_by(var, predictor, value_normed) %>%
arrange(var, predictor, value_normed) %>%
summarise(
    k = n()
) %>%
mutate(
    empirical = cumsum(k) / sum(k),
    theoretical = pnorm(value_normed),
    diff_L2 = k*(empirical - theoretical)**2
) %>%
group_by(var, predictor) %>%
summarise(
    L2_distance = sum(diff_L2)
) %>%
arrange(var, L2_distance) %>%
mutate(
    gain = 100 * (nth(L2_distance, 3) - nth(L2_distance, 1)) / nth(L2_distance, 3)
) %>%
filter(nth(predictor, 1) == 'x' | gain > 20) %>%
filter(row_number() == 1) %>%
arrange(L2_distance)